In [ ]:
# import os
# print("WD: ", os.getcwd())

In [ ]:
from lib.conversions import create_gene_to_protein_mapping


path_file_genes = "../../../resources/Reactome/v70/Genes/all_genes_v70.csv"
create_gene_to_protein_mapping(path_file_genes, "../../../resources/UniProt/", "mapping_proteins_to_genes_v70.tab", 1000)


In [2]:
# Generates module files: gene_modules.tsv, protein_modules.tsv and proteoform_modules.tsv
# Generates single module files at each level: gene, protein and proteoform file for each trait
# Module sizes for genes, protein and proteoform levels
# Variation in module sizes when going from one level to the other
# Three files (for genes, proteins and proteoforms) with the Overlap similarity of each 
# pair of traits.
# Creates a file with a selection of trait pair examples which show decrease in the overlap.
# TODO: Creates a file with examples where the number of crossing edges between modules changed
# TODO: Creates a file with a selection of trait pair examples which show overlap only with 
# modified proteins.

In [3]:
import pandas as pd

scores_genes = pd.read_csv("../../reports/scores_genes_overlap_similarity.tsv", sep='\t')
scores_proteins = pd.read_csv("../../reports/scores_proteins_overlap_similarity.tsv", sep='\t')
scores_proteoforms = pd.read_csv("../../reports/scores_proteoforms_overlap_similarity.tsv", sep='\t')

# scores_genes.index = "Pair"
# scores_genes.rename()
# scores_genes.head()

In [4]:
import pandas as pd

data = pd.read_csv("../../reports/overlap_similarity_score_variation_examples.tsv", sep='\t')
data.index.name = "Trait pair"

data

,TRAIT1,TRAIT2,SCORE_GENES,SCORE_PROTEINS,SCORE_PROTEOFORMS,GENES_TO_PROTEINS,PROTEINS_TO_PROTEOFORMS
Trait pair,,,,,,,
0,Thyrotropin,Waist Circumference,0.166667,0.166667,0.000000,0.0,-0.166667
1,Stroke,Wet Macular Degeneration,0.076923,0.076923,0.000000,0.0,-0.076923
2,Stroke,Triglycerides,0.400000,0.400000,0.375000,0.0,-0.025000
3,Psoriasis,Waist-Hip Ratio,0.111111,0.111111,0.100000,0.0,-0.011111
4,Psoriasis,Waist Circumference,0.029412,0.029412,0.010101,0.0,-0.019311
...,...,...,...,...,...,...,...
877,Electrocardiography,Hip,0.030303,0.030303,0.000000,0.0,-0.030303
878,Electrocardiography,Metabolic Syndrome X,0.500000,0.500000,0.375000,0.0,-0.125000
879,Electrocardiography,Metabolism,0.151515,0.151515,0.108108,0.0,-0.043407


In [5]:
from lib.visualization import plot_module_pair

trait1 = "Thyrotropin"
trait2 = "Waist Circumference"
level = "proteoforms"
path_to_modules = "../../reports/modules/"

plot_module_pair(trait1, trait2, level, path_to_modules)


